In [ ]:
import pandas as pd
import numpy as np
import nltk
import re

In [ ]:
df = pd.read_csv('sample_data/gp-comments-and-responses.csv', 
                 encoding='utf-8"',engine = 'python', encoding_errors='ignore')

In [ ]:
df.head(3)

,Comment Title,Liked,Disliked,Advice,Organisation Response,Unnamed: 5,sentiment
0,The approach of doctors and staff gives confid...,Friendly patient (pun not intended) attitude o...,Waiting times are often quite protracted as on...,Re waiting times - having to wait does suggest...,Thank you for your encouragement! I am one of ...,NaN,Positive
1,"Less Work, More queue",NaN,"They work less hours 4 hours a day as doctors,...",NaN,I'm one of the two full-time doctors at this s...,NaN,Negative
2,Rudeness,I had tried to register at this surgery as my ...,NaN,NaN,Sorry I couldn't respond immediately - I could...,NaN,Negative


In [ ]:
len(df)

1952

In [ ]:
df.isnull().sum()

Comment Title               0
Liked                     383
Disliked                  452
Advice                   1133
Organisation Response       0
Unnamed: 5               1952
sentiment                   0
dtype: int64

In [ ]:
df = df.drop('Liked', axis = 1)

In [ ]:
df = df.drop('Disliked', axis = 1)

In [ ]:
df = df.drop('Advice', axis = 1)

In [ ]:
df = df.drop('Organisation Response', axis = 1)

In [ ]:
df = df.drop('Unnamed: 5', axis = 1)

In [ ]:
df.head()

,Comment Title,sentiment
0,The approach of doctors and staff gives confid...,Positive
1,"Less Work, More queue",Negative
2,Rudeness,Negative
3,"Complacent GP, Misdiagnoses abound",Positive
4,without compassion on the phone.,Negative


In [ ]:
df.columns = ['comment', 'sentiment']

In [ ]:
df.head()

,comment,sentiment
0,The approach of doctors and staff gives confid...,Positive
1,"Less Work, More queue",Negative
2,Rudeness,Negative
3,"Complacent GP, Misdiagnoses abound",Positive
4,without compassion on the phone.,Negative


In [ ]:
df.sentiment.value_counts()

Negative    1008
Positive     677
Neutral      267
Name: sentiment, dtype: int64

In [ ]:
df = df[df.sentiment != "I can't tell"]

In [ ]:
X = df['comment']
y = df['sentiment']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = .2)

In [ ]:
print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Test set size:", len(X_test))

Training set size: 1248
Validation set size: 313
Test set size: 391


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
def ascii_only(text):
    return text.encode("ascii", "ignore").decode()

In [ ]:
def make_lower(text):
    return text.lower()

In [ ]:
# Find links and remove them

def remove_links(text):
    p = '((http\w*:\/\/)?(www\.\w+)?(\w+\.(com|co|ly|ch|org|net)+)(\/\w+)?)'
    return re.sub(p, ' ', text)

In [ ]:
# Remove html symbol
def remove_html(text):
    return re.sub("[^A-Za-z0-9 ]\w+", ' ', text)

In [ ]:
# Remove mentions
def remove_mentions(text):
    return re.sub("(@\w*)", ' ', text)

In [ ]:
# Remove unnecessary punctuations
import string
punctuations = string.punctuation
punctuations = punctuations + '�' + string.digits

def remove_punctuations(text, punctuations):
    table_ = str.maketrans('', '', punctuations)
    return text.translate(table_) 

In [ ]:
def remove_brandwords(text):
    p = '''#?(iphone|ipad|sxsw|hcsm|google|apple|cisco|austin|
    atari|intel|mac|pc|blackberry|android|linux|ubuntu)[a-z0-9]*'''
    return re.sub(p, ' ', text)

In [ ]:
import nltk
nltk.download('stopwords')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

sw = (stopwords.words('english'))

def lemmatize(x, sw):
    wnl = WordNetLemmatizer()
    return ' '.join([wnl.lemmatize(w) for w in x.split() if w not in sw])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocessing(df, punctuations, stopwords):
    df['comment'] = df.comment.apply(lambda x: ascii_only(x))
    df['comment'] = df.comment.apply(lambda x: make_lower(x))
    df['comment'] = df.comment.apply(lambda x: remove_links(x))
    df['comment'] = df.comment.apply(lambda x: remove_html(x))
    df['comment'] = df.comment.apply(lambda x: remove_mentions(x))
    df['comment'] = df.comment.apply(lambda x: remove_punctuations(x, punctuations))
    df['comment'] = df.comment.apply(lambda x: remove_brandwords(x))
    df['comment'] = df.comment.apply(lambda x: lemmatize(x, sw))
    
    return df

In [ ]:
import nltk
nltk.download('wordnet')
dataset = preprocessing(pd.DataFrame(df), punctuations, sw)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
dataset.head(100)

,comment,sentiment
0,approach doctor staff give confidence,Positive
1,le work queue,Negative
2,rudeness,Negative
3,complacent gp misdiagnoses abound,Positive
4,without compassion phone,Negative
...,...,...
95,open hour suit everyone,Positive
96,excellent practice,Positive
97,people practice always respectful address pati...,Positive
98,visit called walk help centre,Negative


In [ ]:
dataset.to_csv(r'preprocessed_gp-comments-and-responses.csv', index=True)